<strong><em>Student Academic Performance Prediction:</em></strong><br>
***
This notebook attempts to study some of the factors affecting student academic success, the first measurement of success being the GPA and the second being eligibility for application of a master's degree

Features being studied are:<br>
<ul> 
<li>SAT score</li>
<li>Athletism</li>
<li>High School Size</li>
<li>High School Rank</li>
<li>High School Percentile</li>
<li>Gender</li>
</ul>
<hr>

<br>

<h3>Libraries Import</h3>
<hr>

UNCOMMENT THE CODE BELOW ONLY IF YOU ARE IMPORTING

In [ ]:
'''
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install statsmodels
%pip install dash
'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import Ridge , LogisticRegression, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import statsmodels.api as sm
from dash import dash, dcc, html, Input, Output
import plotly.express as px


<h3> Data Import</h3>
<p>The main dataframes are:</p> 
<ol>
<li>merged_df</li>
<li>unchanged_df</li>
</ol>
<p>Where <strong>merged_df</strong> is the dataframe we're going to experiment on </p>
<p>And <strong>unchaged_df</strong> is the dataframe used for the plots</p>
<p> A <strong> "Success" </strong> attribute was added to represent student ability to enroll for a masters </p>
<hr>

In [ ]:
merged_df= pd.read_csv('GPA2.csv')
unchanged_df = pd.read_csv('GPA2.csv') # For the "before" graphs


In [ ]:
#adding a "success" column
success =[]
successCount = 0;
failCount = 0;
for i in merged_df["gpa"]:
    if(i >= (3)):
        success.append(True)
        successCount+= 1
    else:
        success.append(False)
        failCount+=1

merged_df.insert(7,"Success", success);
print("Succes Count is: ", successCount)
print("Fail Count is: ", failCount)

  <h3>Exploratory Data Analysis <strong>EDA</strong></h3>
    <p>This part includes:</p>
    <ul>
        <li>Overview of the data</li>
        <li>Graphs for the data</li>
        <li>A dashboard</li>
    </ul>
    <p>Ambiguous or unclear graphs will be explained in the comments.</p>

In [ ]:
unchanged_df

In [ ]:
unchanged_df.describe()

In [ ]:
merged_df.info()

In [ ]:
# we first check for duplicates:
print("Duplicates in data frame:\n",unchanged_df[unchanged_df.duplicated()])
# we found a duplicate in the df, it's student number 533

In [ ]:
merged_df= merged_df.drop_duplicates()
#here we saved the merged_df without the duplicate
merged_df.info()
#we can notice that the data is now one entry less. (it was 4135 and now is 4134)

In [ ]:
# do this again after the duplicate is gone
merged_df.describe()
#we can also conclude a few things from this. for example, nearly half of these students are female, 
#and almost 5% only are athletes

In [ ]:
import plotly.graph_objects as go
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div(style={'backgroundColor': '#f7f7f7', 'padding': '20px'}, children=[
    html.H1("Data Visualization Dashboard", style={'textAlign': 'center', 'margin-bottom': '20px', 'color': '#333'}),
    
    # Plot type selector
    html.Div([
        html.Label("Select Plot Type", style={'font-weight': 'bold', 'color': '#555'}),
        dcc.RadioItems(
            id='plot-type',
            options=[
                {'label': 'Scatter Plot', 'value': 'scatter'},
                {'label': 'Bar Plot', 'value': 'bar'},
                {'label': 'Box Plot', 'value': 'box'},
                {'label': 'Histogram', 'value': 'histogram'},
                {'label': 'Correlation Matrix', 'value': 'correlation'}
            ],
            value='scatter',  # Default value
            labelStyle={'display': 'block'},
            style={'margin-top': '10px'}
        ),
    ], id='controls', style={'width': '50%', 'margin': 'auto', 'textAlign': 'center'}),
    
    # Dropdowns
    html.Div(id='x-variable-div', children=[
        html.Label("Select X Variable", style={'font-weight': 'bold', 'color': '#555'}),
        dcc.Dropdown(
            id='x-variable',
            options=[{'label': col, 'value': col} for col in unchanged_df.columns[1:] if col not in ['athlete', 'female']],
            value=unchanged_df.columns[1],  # Default value
            clearable=False,
            style={'margin-top': '10px'}
        ),
    ], style={'textAlign': 'center', 'display': 'none', 'margin-top': '20px'}),
    
    html.Div(id='box-variable-div', children=[
        html.Label("Select Box Variable", style={'font-weight': 'bold', 'color': '#555'}),
        dcc.Dropdown(
            id='box-variable',
            options=[
                {'label': 'Female', 'value': 'female'},
                {'label': 'Athlete', 'value': 'athlete'}
            ],
            value='female',
            clearable=False,
            style={'margin-top': '10px'}
        ),
    ], style={'textAlign': 'center', 'display': 'none', 'margin-top': '20px'}),
    
    # Graph
    dcc.Graph(id='graph', style={'margin-top': '20px'})
])

# Callbacks
@app.callback(
    [Output('x-variable-div', 'style'), Output('box-variable-div', 'style')],
    Input('plot-type', 'value')
)
def show_hide_dropdowns(plot_type):
    x_style = {'display': 'none'}
    box_style = {'display': 'none'}
    
    if plot_type in ['scatter', 'bar']:
        x_style = {'display': 'block'}
    elif plot_type == 'box':
        box_style = {'display': 'block'}
    
    return x_style, box_style

@app.callback(
    Output('graph', 'figure'),
    [Input('x-variable', 'value'),
     Input('plot-type', 'value'),
     Input('box-variable', 'value')]
)
def update_graph(x_variable, plot_type, box_variable):
    if plot_type == 'scatter':
        fig = px.scatter(unchanged_df, x=x_variable, y='gpa', title=f'GPA vs {x_variable}', labels={'gpa':'GPA Score'})
    elif plot_type == 'bar':
        fig = px.bar(unchanged_df, x=x_variable, y='gpa', title=f'GPA vs {x_variable}', labels={'gpa':'GPA Score'})
    elif plot_type == 'box':
        fig = px.box(unchanged_df, x=box_variable, y='gpa', title=f'GPA vs {box_variable.capitalize()}', labels={'gpa':'GPA Score'})
    elif plot_type == 'histogram':
        bin_num = int(2 * np.cbrt(unchanged_df.size))  # Convert bin_num to an integer
        fig = px.histogram(unchanged_df, x='gpa', title='Histogram of GPA', labels={'gpa': 'GPA Score'}, nbins=bin_num)
    elif plot_type == 'correlation':
        correlation_matrix = unchanged_df[['gpa', 'SAT', 'athlete', 'hsize', 'hsrank', 'hsperc', 'female']].corr()
        correlation_matrix_text = correlation_matrix.round(2).astype(str)
        fig = go.Figure(data=go.Heatmap(x=correlation_matrix.columns, z=correlation_matrix.values, y=correlation_matrix.columns,
            colorscale='RdBu_r', zmin=-1, zmax=1,text=correlation_matrix_text.values,hoverinfo='text'))
        fig.update_traces(text=correlation_matrix_text.values, texttemplate="%{text}")
        fig.update_layout(title='Correlation Matrix', xaxis_nticks=36)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8000)

In [ ]:
#this code is probably going to bye bye
# Generate the correlation matrix
correlation_matrix = merged_df[['gpa', 'SAT', 'athlete','hsize','hsrank','hsperc','female']].corr()

# Generate a heatmap for the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


***
<strong><em>Initial Test Using All of The Dataset:</em></strong><br>
***

In [ ]:
# Split data, we decided to drop hsperc as it is equal to rank/size --> dependent variable
X = merged_df[['SAT', 'athlete','hsize','hsrank','female']]
y = merged_df['gpa']

In [ ]:
#Normal LinearRegression with k = 3 fold cross validation

best_mse = 17 #arbitrarly large MSE, (MSE for 0% accuracy is at maximum 16)
best_model = 0


kf = KFold(n_splits=3, random_state=None)
total_test_mse = 0
total_train_mse = 0
    
for j, (train_index, test_index) in enumerate(kf.split(X)):

    # Train Test split
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    # Normalization
    mean_X_train = np.mean(X_train, axis=0)
    std_X_train = np.std(X_train, axis=0)
    mean_y_train = np.mean(y_train)
    std_y_train = np.std(y_train)


    X_train_norm = (X_train - mean_X_train)/std_X_train
    X_test_norm = (X_test - mean_X_train)/std_X_train

    y_train_norm = (y_train - mean_y_train)/std_y_train
    y_test_norm = (y_test - mean_y_train)/std_y_train

    # Linear Regression
    model = LinearRegression(fit_intercept= True)
    model.fit(X_train_norm, y_train_norm)

    # Train and test MSE
    y_train_predict_unnormalized = (model.predict(X_train_norm) * std_y_train) + mean_y_train
    y_test_predict_unnormalized = (model.predict(X_test_norm) * std_y_train) + mean_y_train

    train_mse = np.mean((y_train_predict_unnormalized - y_train) ** 2)
    test_mse = np.mean((y_test_predict_unnormalized - y_test) ** 2)
    total_train_mse += train_mse
    total_test_mse += test_mse

    if (test_mse < best_mse): #Choose the model with lowest MSE for hypothesis testing and R2 Calculation
        best_mse = test_mse
        best_model = model

print("Average Train MSE:", total_train_mse / kf.get_n_splits())
print("Average Test MSE:", total_test_mse / kf.get_n_splits())
print("Best model coeffecients: ", best_model.coef_)


In [ ]:
#ٌR2 Score Calculation

X_all_norm = (X - mean_X_train)/std_X_train
y_pred_all = best_model.predict(X_all_norm)
y_pred_all_unnormalized = (y_pred_all * std_y_train) + mean_y_train

print ("The r2 score is: ", r2_score(y,y_pred_all_unnormalized))

In [ ]:
#Hypothesis testing on the least error model

X_const_added = sm.add_constant(X_all_norm)
est = sm.OLS(y,X_all_norm)
est2 = est.fit()
print(est2.summary())


In [ ]:
#Plotting a sample of 500 students


numss = [np.random.randint(1,len(y)) for _ in range(500)]
y_rand = y.iloc[numss]
y_pred_rand = y_pred_all_unnormalized[numss]

# Assuming y_pred_all_unnormalized and y are numpy arrays of the same length
plt.figure(figsize=(6, 4))
plt.scatter(y, y, color='blue', label='Predicted', alpha=0.5)
plt.scatter(y, y_pred_all_unnormalized, color='red', label='Actual', alpha=0.5)  # Actual values
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Scatter Plot of Actual vs Predicted')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
best_mse = 10
best_model = 0

alphas = [0, 0.1,0.5, 1,10,100]

for i, alpha in enumerate(alphas):

    kf = KFold(n_splits=3, random_state=None)
    total_test_mse = 0
    total_train_mse = 0
        
    for j, (train_index, test_index) in enumerate(kf.split(X)):

        # Train Test split
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]


        # Normalization
        mean_X_train = np.mean(X_train, axis=0)
        std_X_train = np.std(X_train, axis=0)
        mean_y_train = np.mean(y_train)
        std_y_train = np.std(y_train)


        X_train_norm = (X_train - mean_X_train)/std_X_train
        X_test_norm = (X_test - mean_X_train)/std_X_train

        y_train_norm = (y_train - mean_y_train)/std_y_train
        y_test_norm = (y_test - mean_y_train)/std_y_train

        # Ridge Regression
        model = Ridge(alpha = alpha)
        model.fit(X_train_norm, y_train_norm)

        # Train and test MSE
        y_train_predict_unnormalized = (model.predict(X_train_norm) * std_y_train) + mean_y_train
        y_test_predict_unnormalized = (model.predict(X_test_norm) * std_y_train) + mean_y_train

        train_mse = np.mean((y_train_predict_unnormalized - y_train) ** 2)
        test_mse = np.mean((y_test_predict_unnormalized - y_test) ** 2)
        total_train_mse += train_mse
        total_test_mse += test_mse

        if (test_mse < best_mse):
            best_mse = test_mse
            best_model = model

    print("Average Train MSE:", total_train_mse / kf.get_n_splits())
    print("Average Test MSE:", total_test_mse / kf.get_n_splits())
    print()


In [ ]:
X_all_norm = (X - mean_X_train)/std_X_train
y_pred_all = best_model.predict(X_all_norm)
y_pred_all_unnormalized = (y_pred_all * std_y_train) + mean_y_train

print ("The r2 score is: ", r2_score(y,y_pred_all_unnormalized))

In [ ]:
import matplotlib.pyplot as plt

numss = [np.random.randint(1,len(y)) for _ in range(500)]
y_rand = y.iloc[numss]
y_pred_rand = y_pred_all_unnormalized[numss]

# Assuming y_pred_all_unnormalized and y are numpy arrays of the same length
plt.figure(figsize=(6, 4))
plt.scatter(y, y, color='blue', label='Predicted', alpha=0.5)
plt.scatter(y, y_pred_all_unnormalized, color='red', label='Actual', alpha=0.5)  # Actual values
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Scatter Plot of Actual vs Predicted')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
best_model = 0
best_mse = 10
best_trainmse = 0
# Normalization
mean_X_train = np.mean(X_train, axis=0)
std_X_train = np.std(X_train, axis=0)
mean_y_train = np.mean(y_train)
std_y_train = np.std(y_train)


X_train_norm = (X_train - mean_X_train)/std_X_train
X_test_norm = (X_test - mean_X_train)/std_X_train


y_train_norm = (y_train - mean_y_train)/std_y_train
y_test_norm = (y_test - mean_y_train)/std_y_train

degrees = [1,2,4,8]
alphas = [0.1,0.5,1,5,10]
for deg in degrees:

    # Polynomial Features
    poly = PolynomialFeatures(degree=deg, include_bias= True)  # You can adjust the degree as needed
    X_poly_train_norm = poly.fit_transform(X_train_norm)

    for alpha in alphas:
        model = Ridge(alpha=alpha)
        model.fit(X_poly_train_norm, y_train_norm)

        X_poly_test_norm = poly.fit_transform(X_test_norm)


        train_mse = mean_squared_error(y_train, (model.predict(X_poly_train_norm)*std_y_train)+mean_y_train)
        test_mse = mean_squared_error(y_test, (model.predict(X_poly_test_norm)*std_y_train)+mean_y_train)
        
        if (test_mse < best_mse):
            best_mse = test_mse
            best_trainmse = train_mse
            best_model = model
            best_poly = poly

print("Best model test MSE = ", best_mse)
print("Best model train MSE = ", best_trainmse)

In [ ]:
X_norm = (X - mean_X_train)/std_X_train
X_poly = best_poly.fit_transform(X_norm)
y_pred_all_unnormalized = (best_model.predict(X_poly)*std_y_train)+mean_y_train

print ("The r2 score is: ", r2_score(y,y_pred_all_unnormalized))

numss = [np.random.randint(1,len(y)) for _ in range(500)]
y_rand = y.iloc[numss]
y_pred_rand = y_pred_all_unnormalized[numss]

# Assuming y_pred_all_unnormalized and y are numpy arrays of the same length
plt.figure(figsize=(6, 4))
plt.scatter(y_rand, y_rand, color='blue', label='Predicted', alpha=0.5)
plt.scatter(y_rand, y_pred_rand, color='red', label='Actual', alpha=0.5)  # Actual values
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Scatter Plot of Actual vs Predicted')
plt.legend()
plt.grid(True)
plt.show()
print ("The r2 score is: ", r2_score(y,y_pred_all_unnormalized))


<p> <strong> REMOVING OUTLIERS </strong> </p> <hr>
<p> We used the <strong>Inter Quartile Range (IQR) method</strong> to filter out the outliers </p>


In [ ]:
change = 25

first_quartile = np.percentile(y, change)
fourth_quartile = np.percentile(y, 100-change)

iqr = fourth_quartile - first_quartile

lower_bound = first_quartile - 1.5 * iqr
upper_bound = fourth_quartile + 1.5 * iqr

outlier_indices = np.where((y < lower_bound) | (y > upper_bound))[0]

merged_df_no_OUT = merged_df[(merged_df['gpa'] > lower_bound) & (merged_df['gpa'] < upper_bound)]


X_no_outliers = merged_df_no_OUT[['SAT', 'athlete','hsize','hsrank','hsperc','female']]
y_no_outliers = merged_df_no_OUT['gpa']


In [ ]:
best_mse = 10
best_model = 0


kf = KFold(n_splits=3, random_state=None)
total_test_mse = 0
total_train_mse = 0
    
for j, (train_index, test_index) in enumerate(kf.split(X_no_outliers)):

    # Train Test split
    X_train, X_test = X_no_outliers.iloc[train_index], X_no_outliers.iloc[test_index]
    y_train, y_test = y_no_outliers.iloc[train_index], y_no_outliers.iloc[test_index]


    # Normalization
    mean_X_train = np.mean(X_train, axis=0)
    std_X_train = np.std(X_train, axis=0)
    mean_y_train = np.mean(y_train)
    std_y_train = np.std(y_train)


    X_train_norm = (X_train - mean_X_train)/std_X_train
    X_test_norm = (X_test - mean_X_train)/std_X_train

    y_train_norm = (y_train - mean_y_train)/std_y_train
    y_test_norm = (y_test - mean_y_train)/std_y_train

    # Linear Regression
    model = LinearRegression()
    model.fit(X_train_norm, y_train_norm)

    # Train and test MSE
    y_train_predict_unnormalized = (model.predict(X_train_norm) * std_y_train) + mean_y_train
    y_test_predict_unnormalized = (model.predict(X_test_norm) * std_y_train) + mean_y_train

    train_mse = np.mean((y_train_predict_unnormalized - y_train) ** 2)
    test_mse = np.mean((y_test_predict_unnormalized - y_test) ** 2)
    total_train_mse += train_mse
    total_test_mse += test_mse

    if (test_mse < best_mse):
        best_mse = test_mse
        best_model = model

print("Average Train MSE:", total_train_mse / kf.get_n_splits())
print("Average Test MSE:", total_test_mse / kf.get_n_splits())


In [ ]:
X_all_norm = (X_no_outliers - mean_X_train)/std_X_train
y_pred_all = best_model.predict(X_all_norm)
y_pred_all_unnormalized = (y_pred_all * std_y_train) + mean_y_train

print ("The r2 score is: ", r2_score(y_no_outliers,y_pred_all_unnormalized))

In [ ]:
numss = [np.random.randint(1,len(y_no_outliers)) for _ in range(500)]
y_rand = y_no_outliers.iloc[numss]
y_pred_rand = y_pred_all_unnormalized[numss]

# Assuming y_pred_all_unnormalized and y are numpy arrays of the same length
plt.figure(figsize=(6, 4))
plt.scatter(y_rand, y_rand, color='blue', label='Predicted', alpha=0.5)
plt.scatter(y_rand, y_pred_rand, color='red', label='Actual', alpha=0.5)  # Actual values
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Scatter Plot of Actual vs Predicted')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_no_outliers, y_no_outliers, test_size=0.2, random_state=50)
best_model = 0
best_mse = 10
best_trainmse = 0

# Normalization
mean_X_train = np.mean(X_train, axis=0)
std_X_train = np.std(X_train, axis=0)
mean_y_train = np.mean(y_train)
std_y_train = np.std(y_train)


X_train_norm = (X_train - mean_X_train)/std_X_train
X_test_norm = (X_test - mean_X_train)/std_X_train


y_train_norm = (y_train - mean_y_train)/std_y_train
y_test_norm = (y_test - mean_y_train)/std_y_train

degrees = [1,2,4,8]
alphas = [0.1,0.5,1,5,10]
for deg in degrees:

    # Polynomial Features
    poly = PolynomialFeatures(degree=deg, include_bias= True)  # You can adjust the degree as needed
    X_poly_train_norm = poly.fit_transform(X_train_norm)

    for alpha in alphas:
        model = Ridge(alpha=alpha)
        model.fit(X_poly_train_norm, y_train_norm)

        X_poly_test_norm = poly.fit_transform(X_test_norm)


        train_mse = mean_squared_error(y_train, (model.predict(X_poly_train_norm)*std_y_train)+mean_y_train)
        test_mse = mean_squared_error(y_test, (model.predict(X_poly_test_norm)*std_y_train)+mean_y_train)
        
        if (test_mse < best_mse):
            best_mse = test_mse
            best_trainmse = train_mse
            best_model = model
            best_poly = poly

print("Best model test MSE = ", best_mse)
print("Best model train MSE = ", best_mse)

In [ ]:
X_norm = (X_no_outliers - mean_X_train)/std_X_train
X_poly = best_poly.fit_transform(X_norm)
y_pred_all_unnormalized = (best_model.predict(X_poly)*std_y_train)+mean_y_train

print ("The r2 score is: ", r2_score(y_no_outliers,y_pred_all_unnormalized))


In [ ]:
numss = [np.random.randint(1,len(y_no_outliers)) for _ in range(500)]
y_rand = y_no_outliers.iloc[numss]
y_pred_rand = y_pred_all_unnormalized[numss]

# Assuming y_pred_all_unnormalized and y are numpy arrays of the same length
plt.figure(figsize=(6, 4))
plt.scatter(y_rand, y_rand, color='blue', label='Predicted', alpha=0.5)
plt.scatter(y_rand, y_pred_rand, color='red', label='Actual', alpha=0.5)  # Actual values
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Scatter Plot of Actual vs Predicted')
plt.legend()
plt.grid(True)
plt.show()

The start of <strong>Classification</strong> code cleaning.

In [ ]:
X = merged_df[['SAT', 'athlete','hsize','hsrank','female']] #Same concept here, dropping feature hs percentile as it is dependent on rank and size
y = merged_df['Success']

# Train-test split
def KNNClassification(k):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
    model = KNeighborsClassifier(n_neighbors= k)

    #Normalization
    mean_X_train = np.mean(X_train, axis=0)
    std_X_train = np.std(X_train, axis=0)


    X_train_norm = (X_train-mean_X_train)/std_X_train


    model.fit(X_train_norm,y_train)

    X_test_norm = (X_test - mean_X_train)/ std_X_train


    y_pred = model.predict(X_test_norm)

    cm = confusion_matrix(y_test, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Fail", "Success"],)
    cm_display.plot(cmap = "YlOrRd")
    plt.show()

    print(model.score(X_test_norm, y_test))

In [ ]:
def LogisticRegressionClassification(alphaValue):
    model = LogisticRegression(random_state=0, C=alphaValue,fit_intercept=True)
    model.fit(X_train_norm, y_train)

    y_pred = model.predict(X_test_norm)
    cm = confusion_matrix(y_test, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Fail", "Success"],)
    cm_display.plot(cmap = "BuPu")
    plt.show()

    print(model.score(X_test_norm, y_test))

In [ ]:
def supportVectorClassifier():
    model = SVC(kernel='rbf', probability=True)
    model.fit(X_train_norm, y_train)

    y_pred = model.predict(X_test_norm)
    cm = confusion_matrix(y_test, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Fail", "Success"],)
    cm_display.plot(cmap = "binary")
    plt.show()

    print(model.score(X_test_norm, y_test))


In [ ]:
# Dashboard for Classification
import plotly.figure_factory as ff


fontColor = '#FFFFFF'

app = dash.Dash(__name__)
flag = 1

app.layout = html.Div([
    html.H1(
        "Classification Confusion Matrices Dashboard",
        style={'text-align': 'center'}
    ),
    dcc.Dropdown(
        id='classification-model',
        options=[
            {'label': 'KNN', 'value': 'KNN'},
            {'label': 'LogRegression', 'value': 'LogRegression'},
            {'label': 'SVC', 'value': 'SVC'}
        ],
        value='KNN',
        style={'width':'500px'}
    ),
    html.Div([
        html.H5("N = "),
        dcc.Input(id='modelProp', value=5, type='text')
    ], className='KNN-div',
    style={'display': 'flex', 'flex-direction': 'row', 'gap': '10px','height':'30px','align-items':'center'}),

    dcc.Graph(id='graph',style={'width':'700px'}),
    html.Div(id='score')
],style={'display':'flex','align-items':'center','justify-content':'center','flex-direction':'column','background-color':'white','gap':'10px'})

@app.callback(
    [Output('graph', 'figure'),
    Output('score','children')],
    [Input('classification-model', 'value'),
     Input('modelProp','value')]
)

def update(model, modelProp):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
    n = 5

    if model == 'KNN':
        if modelProp != '':
            n = int(modelProp)
        mymodel = KNeighborsClassifier(n_neighbors=n)

    elif model == 'LogRegression':
        if modelProp != '':
            n = float(modelProp)
            if n != 0:
                mymodel = LogisticRegression(random_state=0, C=n, fit_intercept=True)
            else:
                mymodel = LogisticRegression(random_state=0, C=0.01, fit_intercept=True)
        else:
            mymodel = LogisticRegression(random_state=0, C=0.01, fit_intercept=True)

    elif model == 'SVC':
        mymodel = SVC(kernel='rbf', probability=True)

    # Normalization
    mean_X_train = np.mean(X_train, axis=0)
    std_X_train = np.std(X_train, axis=0)

    X_train_norm = (X_train - mean_X_train) / std_X_train

    mymodel.fit(X_train_norm, y_train)

    X_test_norm = (X_test - mean_X_train) / std_X_train

    y_pred = mymodel.predict(X_test_norm)

    cm = confusion_matrix(y_test, y_pred)

    x_values = ["Fail", "Success"]
    y_values = x_values[::].copy()
    cm_text = [[str(y_values) for y_values in x_values] for x_values in cm]
    fig = ff.create_annotated_heatmap(cm, x=x_values, y=y_values, annotation_text=cm_text, colorscale='Viridis')

    fig.update_layout(title_text='<b>Confusion Matrix</b>')

    fig.add_annotation(dict(font=dict(color="black", size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black", size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))
    
    # adjust margins to make room for yaxis title
    fig.update_layout(margin=dict(t=50, l=200))
    
    
    tn, fp, fn, tp = cm.ravel()

    Recall = tp/(tp+fn)
    Precision = tp/(tp+fp)
    Accuracy = (tp+tn)/(tp+fn+tn+fp)
    F_measure = (2*Recall*Precision)/(Recall+Precision)

    return fig,Accuracy


if __name__ == '__main__':
    app.run_server(debug=True, port=8080)